--- Day 15: Beacon Exclusion Zone ---

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable **sensors** that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source **beacon**. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can **determine the position of a beacon precisely**; however, sensors can only lock on to the one beacon **closest to the sensor** as measured by the [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry). (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:
```
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
```
So, consider the sensor at `2,18`; the closest beacon to it is at `-2,15`. For the sensor at `9,16`, the closest beacon to it is at `10,16`.

Drawing sensors as `S` and beacons as `B`, the above arrangement of sensors and beacons looks like this:
```
               1    1    2    2
     0    5    0    5    0    5
 0 ....S.......................
 1 ......................S.....
 2 ...............S............
 3 ................SB..........
 4 ............................
 5 ............................
 6 ............................
 7 ..........S.......S.........
 8 ............................
 9 ............................
10 ....B.......................
11 ..S.........................
12 ............................
13 ............................
14 ..............S.......S.....
15 B...........................
16 ...........SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
```
This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at 8,7:
```
               1    1    2    2
     0    5    0    5    0    5
-2 ..........#.................
-1 .........###................
 0 ....S...#####...............
 1 .......#######........S.....
 2 ......#########S............
 3 .....###########SB..........
 4 ....#############...........
 5 ...###############..........
 6 ..#################.........
 7 .#########S#######S#........
 8 ..#################.........
 9 ...###############..........
10 ....B############...........
11 ..S..###########............
12 ......#########.............
13 .......#######..............
14 ........#####.S.......S.....
15 B........###................
16 ..........#SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
```
This sensor's closest beacon is at `2,10`, and so you know there are no beacons that close or closer (in any positions marked `#`).

None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it **isn't**. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where `y=10`, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:
```
                 1    1    2    2
       0    5    0    5    0    5
 9 ...#########################...
10 ..####B######################..
11 .###S#############.###########.
```

In this example, in the row where `y=10`, there are **`26`** positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. **In the row where `y=2000000`, how many positions cannot contain a beacon?**

In [1]:
# Data Load
example_lines = [
    "Sensor at x=2, y=18: closest beacon is at x=-2, y=15",
    "Sensor at x=9, y=16: closest beacon is at x=10, y=16",
    "Sensor at x=13, y=2: closest beacon is at x=15, y=3",
    "Sensor at x=12, y=14: closest beacon is at x=10, y=16",
    "Sensor at x=10, y=20: closest beacon is at x=10, y=16",
    "Sensor at x=14, y=17: closest beacon is at x=10, y=16",
    "Sensor at x=8, y=7: closest beacon is at x=2, y=10",
    "Sensor at x=2, y=0: closest beacon is at x=2, y=10",
    "Sensor at x=0, y=11: closest beacon is at x=2, y=10",
    "Sensor at x=20, y=14: closest beacon is at x=25, y=17",
    "Sensor at x=17, y=20: closest beacon is at x=21, y=22",
    "Sensor at x=16, y=7: closest beacon is at x=15, y=3",
    "Sensor at x=14, y=3: closest beacon is at x=15, y=3",
    "Sensor at x=20, y=1: closest beacon is at x=15, y=3",
]

with open("day_15_input.txt") as _file:
    data_lines = _file.readlines()

In [2]:
from __future__ import annotations
import re

class Vector:
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y

    def __members(self):
        return self.x, self.y

    def __repr__(self):
        return str(self.__members())

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __mul__(self, other: int):
        return Vector(self.x * other, self.y * other)

    def __eq__(self, other):
        return self.__members() == other.__members()

    def __hash__(self):
        return hash(self.__members())

    def dist_manhattan(self, other: Vector):
        dist = self - other
        return abs(dist.x) + abs(dist.y)

    def copy(self):
        return Vector(self.x, self.y)

    @classmethod
    def zero(cls):
        return cls(0, 0)

    @classmethod
    def one(cls):
        return cls(1, 1)


class Sensor:
    _INTEGER_PATTERN = r"[-+]?\d+"
    _PARSE_REGEX = re.compile(
        rf"^Sensor at x=(?P<sensor_x>{_INTEGER_PATTERN}), y=(?P<sensor_y>{_INTEGER_PATTERN}): closest beacon is at x=(?P<beacon_x>{_INTEGER_PATTERN}), y=(?P<beacon_y>{_INTEGER_PATTERN})")

    def __init__(self, position: Vector, beacon: Vector):
        self.position = position
        self.beacon = beacon
        self._dist_manhattan = position.dist_manhattan(beacon)

    def _get_diamond(self, size):
        directions = [
            Vector(1, 1),
            Vector(-1, 1),
            Vector(-1, -1),
            Vector(1, -1)
        ]

        current = self.position + Vector(0, -size)

        for direction in directions:
            for i in range(size):
                current += direction
                yield current

    def __repr__(self):
        return f"{self.position}, {self.beacon}, {self.dist_manhattan}"

    @classmethod
    def parse(cls, line):
        match = cls._PARSE_REGEX.search(line)

        sensor = Vector(int(match.group("sensor_x")), int(match.group("sensor_y")))
        beacon = Vector(int(match.group("beacon_x")), int(match.group("beacon_y")))

        return cls(sensor, beacon)

    @property
    def dist_manhattan(self):
        return self._dist_manhattan

    @property
    def exterior(self):
        return self._get_diamond(self.dist_manhattan + 1)

    def contains(self, tile: Vector):
        return self.position.dist_manhattan(tile) <= self.dist_manhattan

    def get_covered_area_at_row(self, row: int):
        covered_positions = set()

        dist_to_row = abs(self.position.y - row)

        if dist_to_row <= self.dist_manhattan:
            for x in range(dist_to_row - self.dist_manhattan, self.dist_manhattan - dist_to_row + 1):
                position = Vector(self.position.x + x, row)
                covered_positions.add(position)
        return covered_positions


class Grid:
    def __init__(self, sensors: list[Sensor]):
        self._sensors = sensors
        self._beacons = set([x.beacon for x in sensors])

    @classmethod
    def parse(cls, lines):
        sensors = [Sensor.parse(line) for line in lines]
        return cls(sensors)

    def get_covered_areas_at_row(self, row: int):
        covered_areas = set()
        for sensor in self._sensors:
            covered_areas.update(sensor.get_covered_area_at_row(row))

        return len([x for x in covered_areas if x not in self._beacons])

    def get_uncovered_position(self, lower, upper):
        for sensor in self._sensors:
            for tile in sensor.exterior:
                if tile.x < lower.x or tile.x > upper.x or tile.y < lower.y or tile.y > upper.y:
                    continue

                any_contains = False
                for target in self._sensors:
                    if sensor.position == target.position:
                        continue

                    if target.contains(tile):
                        any_contains = True
                        break

                if not any_contains:
                    return tile



In [3]:
# Solution 1 Code
def solution_1(lines, row):
    grid = Grid.parse(lines)
    return grid.get_covered_areas_at_row(row)

def test_solution_1():
    actual = solution_1(example_lines, 10)
    expected = 26
    
    print(actual, expected)
    assert actual == expected
test_solution_1()

26 26


In [4]:
# Solution 1
solution_1(data_lines, 2_000_000)

4873353

--- Part Two ---

Your handheld device indicates that the distress signal is coming from a beacon nearby. The distress beacon is not detected by any sensor, but the distress beacon must have `x` and `y` coordinates each no lower than `0` and no larger than `4000000`.

To isolate the distress beacon's signal, you need to determine its **tuning frequency**, which can be found by multiplying its `x` coordinate by `4000000` and then adding its `y` coordinate.

In the example above, the search space is smaller: instead, the `x` and `y` coordinates can each be at most `20`. With this reduced search area, there is only a single position that could have a beacon: `x=14, y=11`. The tuning frequency for this distress beacon is **`56000011`**.

Find the only possible position for the distress beacon. **What is its tuning frequency?**

In [6]:
# Solution 2 Code
def solution_2(lines, size:int):
    grid = Grid.parse(lines)

    bounds = [Vector(0, 0), Vector(size, size)]

    position = grid.get_uncovered_position(*bounds)
    print(position)

    return position.x * 4_000_000 + position.y
    
    

def test_solution_2():
    actual = solution_2(example_lines, 20)
    expected = 56000011
    
    print(actual, expected)
    assert actual == expected
test_solution_2()

(14, 11)
56000011 56000011


In [7]:
# Solution 2
solution_2(data_lines, 4_000_000)

(2900205, 3139120)


11600823139120